# Tidal Analysis
## Group Members: Burleigh Charlton
This notebook uses tidal measurements taken in Santa Cruz, California between in January and February 2022 and fits an oscilatorry function to it the data. Subsequent minor data analysis is done on the fit and a tsunami outlier is considered

## Import Libraries and set plot style

In [ ]:
from scipy import optimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 120

## Import Data and Prep for analysis

In [ ]:
colnames=['day', 'time', 'tide'] 
df = pd.read_fwf(r'ASTR19_S22_group_project_data.txt', header=None, skiprows=(0,1,2), names=colnames)
df.head()

### Convert Conventional Time to Float

In [ ]:
def timeto_min(ts):
  '''
  takes a string representing the time and returns minutes from start of day
  '''
  minutes = sum(int(x) * 60 ** i for i, x in enumerate(reversed(ts.split(':'))))
  return minutes

In [ ]:
h= df['day']
h = 24 * 60 * h
ti= df['time']
ti = np.array(list(map(timeto_min, ti)))

In [ ]:
#After looking at the results created by minutes I believe that hours may be a more suitable metric
x = h+ti
x = x/60
#x now represents hours

In [ ]:
#Check that the data is monotonically increasing via a plot
plt.plot(x)
plt.title('Time In Hours')

In [ ]:
#and now we store it in the dataframe
df['hours'] = x

In [ ]:
max(x)

In [ ]:
df['y_error'] = np.full(len(df['tide']), .25)

### Initial Plot of Data

In [ ]:
plt.plot( df['hours'], df['tide'], label = 'raw data')
plt.xlabel('time in hours from year start')
plt.ylabel('tide height in feet')
plt.title('Raw Data')
plt.legend()
plt.savefig('data.png')

## Curve Fitting

In [ ]:
#here we create an interpolated set of time data to plot our model against
hours_i = np.linspace(1,1000,10000)

This objecive function is of the form $(a_1 \cdot \sin(f_1 \cdot x + p_1 ) + a_2\cdot \sin(f_2 \cdot x +p_2) + h_{offset}) \cdot a_o(\sin(f_o \cdot x)  + p_o)  + v_{offset}$ where $a$s are amplitudes, $p$s are phases, and $f$s are freqiencies.  
This is designed such that I sum two freqencies with around 2 week and 4 week periods to represent the oscillating amplitude, on the 24 hour tidal changes. 
This is reflected in my inital guesses

In [ ]:
#with another amplitude parameter
##now lets try and make it have a sinusoidal wave as an amplitude to get intra day variation
#(a1 * sin(f1 * x + p1 ) + a2* sin(f2 * x +p2)  * (sin(fo * x)  + po) + vertical_offsetmean
def three_sine(x, a1, f1,p1, a2, f2,p2, hoff,ao,fo,po,v):
  return  (a1 * np.sin(f1*x + p1) + a2* np.sin(f2*x + p2) + hoff) * ao *(np.sin(fo*x + po)) + v
params, paramscov = optimize.curve_fit(three_sine,x, df['tide'], p0=[1.5,(2*np.pi/730),4,6,(2*np.pi/336),2,1,3,(2*np.pi/24),3.22,2], sigma=df['y_error'])#p0=[1,(2*np.pi/730),1,1,(2*np.pi/336),1,(2*np.pi/24),2.22,2]

a1_fit   = params[0]
f1_fit   = params[ 1]
p1_fit   = params[2 ]
a2_fit   = params[ 3]
f2_fit   = params[4 ]
p2_fit   = params[5]
hoff_fit = params[6 ]
ao_fit   = params[ 7]
fo_fit   = params[8]
po_fit   = params[ 9]
v_fit    = params[10]

print(params)
omodel_fit =  ( a1_fit * np.sin(f1_fit*x + p1_fit) +  a2_fit * np.sin(f2_fit*x + p2_fit) + hoff_fit) * ao_fit* (np.sin(fo_fit*x + po_fit)) + v_fit
df['omodel_fit'] = omodel_fit
omodel_i_fit = ( a1_fit * np.sin(f1_fit*hours_i + p1_fit) +  a2_fit * np.sin(f2_fit*hours_i + p2_fit) + hoff_fit) *ao_fit* (np.sin(fo_fit*hours_i + po_fit)) + v_fit

In [ ]:
#Here we plot the model against our smooth representative time space
plt.plot(hours_i, omodel_i_fit)
plt.title('Oscillatory Model')
plt.xlabel('time in hours from year start')
plt.ylabel('tide height in feet')
plt.savefig('fit.png')

## Fit Plot against Data

In [ ]:
#f = plt.figure(figsize = (7,7))
#plt.errorbar(df['hours'],df['tide'], yerr = df['y_error'], fmt = 'o', label = 'data with error')
plt.plot(df['hours'],df['tide'], label = 'data')
plt.plot(df['hours'],omodel_fit, label = 'oscillatory model fit')
#plt.plot(x,tide_fit, label = 'tide_fit')
plt.xlabel('time in hours from year start')
plt.ylabel('tide height in feet')
plt.legend(loc = 'upper right' )
plt.title('Data and Model')
plt.savefig('fitvdata.png')

In [ ]:
#f = plt.figure(figsize = (7,7))
plt.errorbar(df['hours'],df['tide'], yerr = df['y_error'], fmt = 'o', label = 'data with error')
#plt.plot(df['hours'],df['tide'], label = 'data')
plt.plot(df['hours'],omodel_fit, label = 'oscillatory fit')
#plt.plot(x,tide_fit, label = 'tide_fit')
plt.xlabel('time in hours from year start')
plt.ylabel('tide height in feet')
leg = plt.legend(loc = 'upper right', frameon = True )
leg.get_frame().set_edgecolor('b')
plt.title('Data and Model With Expected Error')
plt.savefig('fitvdata.png')

## Residual Analysis

In [ ]:
df['residuals'] = df['tide'] - df['omodel_fit']

In [ ]:
df['residuals'].describe()

In [ ]:
plt.plot(df['hours'],df['residuals'])
plt.xlabel('time in hours from year start')
plt.ylabel('Residual Error')
#leg = plt.legend(loc = 'upper right', frameon = True )
leg.get_frame().set_edgecolor('b')
plt.title('Residuals over time')
plt.savefig('ResidualPlot.png')

In [ ]:
plt.hist(df['residuals'], bins = 14, label = 'residuals to data', alpha=0.7, rwidth=0.85)
plt.xlabel('Residual Error')
plt.ylabel('Frequency')
plt.text(.6, 10, r'$\mu=0, \sigma=0.45$')
plt.title('Histogram of Residuals')
plt.legend()
plt.savefig('ResidualHist.png')

In my fit the residuals are very centered around 0 indicating that the the offset is well calibrated, but there are signfigant errors in the actual fit. With a standard deviation of .45 we are worse off than the expected inaccuracies in our data. The maximum residual found was 1.  



## Hunga Tonga-Hunga Ha'apai Volcano Eruption Tsunami Analysis
Here we add an outlier of a 2ft swell to our model we can see how unlikely this is

In [ ]:
plt.hist(pd.concat([df['residuals'], pd.Series([2])]), bins = 14, label = 'residuals to data', alpha=0.7, rwidth=0.85)
plt.xlabel('Residual Error')
plt.ylabel('Frequency')
#plt.text(.6, 10, r'$\mu=0, \sigma=0.45$')
plt.title('Histogram of Residuals with Outlier')
plt.legend()
plt.savefig('ResidualHist_outlier.png')

To calculate how unlikely this event is we can check the z score via $\frac{x - \mu}{\sigma}$

In [ ]:
zscore = (2 - df['residuals'].mean())/df['residuals'].std()
print(zscore)

So with a z-score of 4.45 we can calculate the probability that this is an outlier or that it is inherent in the model

In [ ]:
from scipy.special import erf
zf = lambda zs: (.5*(1+ erf(zs/2 **.5)))
1 -(zf(zscore) - zf(-1*zscore))

Assuming a gaussian distribution of residuals, the chance that this tide occurs naturally is  8.986910277508642e-06
So we can say with 99.999% confidence that this is an outlier in our model.